In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.
C:\Users\tahas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tahas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tahas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tahas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
env = gym.make('CartPole-v0')
c = env.observation_space.low
c

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

In [3]:
ENV_NAME ='CartPole-v0'

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

In [8]:
class DQN_Solver:
    def __init__(self,observation_space,action_space):
        self.exploration_rate = EXPLORATION_MAX
        self.observation_space = observation_space
        self.memory = deque(maxlen=MEMORY_SIZE)
        
        self.model = Sequential()
        self.model.add(Dense(24, input_shape = (self.observation_space,),activation = 'relu'))
        self.model.add(Dense(24,activation = 'relu'))
        self.model.add(Dense(action_space, activation ='linear'))
        self.model.compile(loss = 'mse' , optimizer = Adam(lr= LEARNING_RATE))
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        if np.random.rand()<= self.exploration_rate: return random.randrange(env.action_space.n)
        mod = self.model.predict(state)
        return np.argmax(mod[0])
        
    def experience(self):
        if len(self.memory)<BATCH_SIZE: return
        batch = random.sample(self.memory,BATCH_SIZE)
        for state,action,reward,next_state,done in batch:
            q_update = reward
            if not done:
                q_update = (reward + GAMMA * np.argmax(self.model.predict(next_state)[0]))
            q_value = self.model.predict(state)
            q_value[0][action] = q_update
            self.model.fit(state,q_value,verbose= 0)
            self.exploration_rate *= EXPLORATION_DECAY
            self.exploration_rate = max(self.exploration_rate, EXPLORATION_MIN )
            

In [ ]:
env = gym.make(ENV_NAME)

action_space = env.action_space.n

observation_space = env.observation_space.shape[0]

dqn = DQN_Solver(observation_space, action_space)

while True:
    state = env.reset()
    state = np.reshape(state,[1,observation_space])
    
    while True:
        env.render()
        action  = dqn.act(state)
        new_state,reward,done,_ = env.step(action)
        new_state = np.reshape(new_state,[1, observation_space])
        dqn.remember(state,action,reward,new_state,done)
        state = new_state
        if done:
                print ("exploration: " + str(dqn.exploration_rate))
                break
        dqn.experience()